In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("roneneldan/TinyStories")

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

In [3]:
for i in dataset["train"]:
    print(i["text"])
    break

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.


In [4]:
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
vocab_size =tokenizer.vocab_size
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 50257


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

# Function to create a causal mask
def generate_causal_mask(seq_len):
    mask = torch.tril(torch.ones(seq_len, seq_len))  # Lower triangular matrix
    return mask


class GPT(nn.Module):
    def __init__(self, vocab_size, context_size, embedd_dim, num_heads):
        super().__init__()
        self.embedd_dim = embedd_dim  #
        self.embedding_token = nn.Embedding(vocab_size, embedd_dim)
        self.embedding_pos = nn.Embedding(context_size, embedd_dim)
        self.trf_blocks = nn.ModuleList(
            [TransformerBlock(num_heads=num_heads, emb_dim=embedd_dim) for _ in range(2)]
        )
        self.final_linear = nn.Linear(embedd_dim, vocab_size)  # Match embedding size

    def forward(self, x, mask):
        batch_size, seq_len = x.shape
        tok_embeds = self.embedding_token(x)
        pos_embeds = self.embedding_pos(torch.arange(seq_len, device=x.device))
        x = tok_embeds + pos_embeds  
        for block in self.trf_blocks:
            x = block(x, mask)  
        logits = self.final_linear(x)
        return logits



class TransformerBlock(nn.Module):
    def __init__(self, num_heads, emb_dim):
        super().__init__()
        self.multi_head = MultiHeadAttention(num_heads, emb_dim)
        self.norm_1 = LayerNorm(emb_dim=emb_dim)
        self.norm_2 = LayerNorm(emb_dim=emb_dim)
        self.feed_nn = FeedForwardNN(emb_dim)

    def forward(self, x, mask):
        skip_con1 = x
        x = self.multi_head(x, mask)
        x = self.norm_1(skip_con1 + x)

        skip_con2 = x
        x = self.feed_nn(x)  # Incorrect: was `self.FeedForwardNN(x)`
        x = self.norm_2(skip_con2 + x)
        return x
        

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, emb_dim):
        super().__init__()
        self.heads = nn.ModuleList(
            [SelfAttention(emb_dim, emb_dim) for _ in range(num_heads)]
        )
        self.out_linear = nn.Linear(emb_dim * num_heads, emb_dim) 

    def forward(self, x, mask):
        context_vec = torch.cat([head(x, mask) for head in self.heads], dim=-1)
        return self.out_linear(context_vec)
        

class SelfAttention(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.query = nn.Linear(input_size, output_size)
        self.key = nn.Linear(input_size, output_size)
        self.value = nn.Linear(input_size, output_size)
    
    def forward(self, x, mask):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (k.shape[-1] ** 0.5)  # Fixed transpose
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, float('-inf'))
        
        attn_probs = torch.softmax(attn_scores, dim=-1)  # Fixed double scaling
        out = torch.matmul(attn_probs, v)
        return out
        
class FeedForwardNN(nn.Module):
    def __init__(self, emb_dim, hidden_neurons=128):
        super().__init__()
        self.layer1 = nn.Linear(emb_dim, hidden_neurons)
        self.layer2 = nn.Linear(hidden_neurons, emb_dim)  # Ensure output matches emb_dim
        self.activation = GELU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        return x

        

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from transformers import GPT2TokenizerFast
import torch
from torch.utils.data import Dataset, DataLoader

class GPTStories(Dataset):
    def __init__(self, texts, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        for txt in texts:  # Iterate through multiple documents
            token_ids = tokenizer.encode(txt, max_length=1024, truncation=True)

            for i in range(0, len(token_ids) - max_length, stride):
                input_chunk = token_ids[i:i + max_length]
                target_chunk = token_ids[i + 1: i + max_length + 1]
                self.input_ids.append(torch.tensor(input_chunk))
                self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
        
def create_dataloader(texts, batch_size=4, max_length=256, stride=128, shuffle=True):
    
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    
    dataset = GPTStories(texts, tokenizer, max_length, stride)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return dataloader

# ====== Causal Mask Generation ======
def generate_causal_mask(seq_len):
    return torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)  # Shape (1, seq_len, seq_len)

# ====== Training Loop ======
def train(model, dataloader, optimizer, criterion, num_epochs=5, device="cuda"):
    model.to(device)
    
    for epoch in range(num_epochs):
        total_loss = 0
        model.train()

        for batch_idx, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)
            mask = generate_causal_mask(x.shape[1]).to(device)

            optimizer.zero_grad()
            logits = model(x, mask)  # Forward pass
            loss = criterion(logits.view(-1, logits.shape[-1]), y.view(-1))  # Cross-entropy loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# ====== Model Initialization ======
vocab_size = 50257
embedding_dim = 128 
batch_size = 128
max_length = 32
num_heads = 4
sample_data = [i["text"] + "<|endoftext|>" for i in dataset["validation"]]

dataloader = create_dataloader(
    sample_data,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)

model = GPT(vocab_size=vocab_size, context_size=max_length, embedd_dim=embedding_dim, num_heads=num_heads)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()



In [16]:
model

GPT(
  (embedding_token): Embedding(50257, 128)
  (embedding_pos): Embedding(32, 128)
  (trf_blocks): ModuleList(
    (0-1): 2 x TransformerBlock(
      (multi_head): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x SelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
          )
        )
        (out_linear): Linear(in_features=512, out_features=128, bias=True)
      )
      (norm_1): LayerNorm()
      (norm_2): LayerNorm()
      (feed_nn): FeedForwardNN(
        (layer1): Linear(in_features=128, out_features=128, bias=True)
        (layer2): Linear(in_features=128, out_features=128, bias=True)
        (activation): GELU()
      )
    )
  )
  (final_linear): Linear(in_features=128, out_features=50257, bias=True)
)

In [17]:
def print_model_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}\n")

    # Print individual layer parameter sizes
    # for name, param in model.named_parameters():
    #     print(f"{name}: {param.size()} -> {param.numel()} params")

# Example usage
print_model_params(model)

Total Parameters: 13,514,833
Trainable Parameters: 13,514,833



In [18]:
# Train the model
train(model, dataloader, optimizer, criterion, num_epochs=10, device="cuda" if torch.cuda.is_available() else "cpu")


Epoch 1/10, Loss: 4.7566
Epoch 2/10, Loss: 3.5525
Epoch 3/10, Loss: 3.2829
Epoch 4/10, Loss: 3.1303
Epoch 5/10, Loss: 3.0274
Epoch 6/10, Loss: 2.9512
Epoch 7/10, Loss: 2.8911
Epoch 8/10, Loss: 2.8421
Epoch 9/10, Loss: 2.8007
Epoch 10/10, Loss: 2.7649


In [23]:
sample_data = [i["text"] + "<|endoftext|>" for idx, i in enumerate(dataset["train"]) if idx<100]

test_dataloader = create_dataloader(
    sample_data,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)

In [47]:
import torch
from transformers import GPT2TokenizerFast

def logits_to_words(input_tokens, logits, tokenizer, idx):
    """
    Convert logits to words by selecting the highest probability tokens and appending to input.

    Args:
        input_tokens (torch.Tensor): Current input tokens of shape (1, seq_len)
        logits (torch.Tensor): Model output logits of shape (1, seq_len, vocab_size)
        tokenizer (GPT2TokenizerFast): Tokenizer to decode tokens
        idx (int): Index of the current word in generation

    Returns:
        torch.Tensor: Updated input tokens with predicted tokens appended
    """
    # Get the most probable token index
    predicted_id = torch.argmax(logits[:, -1, :], dim=-1).item()  # Get scalar value

    # Ensure token index is valid
    if predicted_id >= tokenizer.vocab_size:
        raise ValueError(f"Invalid token index detected (out of vocab range)")

    # Decode tokens into words
    predicted_word = tokenizer.decode(predicted_id, skip_special_tokens=True)
    current_text = tokenizer.decode(input_tokens.squeeze(0), skip_special_tokens=True)

    # Append predicted token to input
    return torch.cat([input_tokens, torch.tensor([[predicted_id]])], dim=-1)

# Setup
device = 'cpu'
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model.to(device)

def generate_text(model, tokenizer, input_tokens, num_words=5):
    """Generates next words for a single sentence (batch size = 1)."""
    input_tokens = input_tokens.to(device)
    prev_tokens = input_tokens

    for i in range(num_words):
        # Ensure input_tokens shape remains (1, seq_len)
        input_tokens = input_tokens[:, -32:]  # Keep last 64 tokens if sequence grows
        mask = generate_causal_mask(input_tokens.shape[1]).to(device)
        logits = model(input_tokens, mask)  # Forward pass
        input_tokens = logits_to_words(input_tokens, logits, tokenizer, i)

    return tokenizer.decode(prev_tokens.squeeze(0), skip_special_tokens=True) +" (pred started here) "+ tokenizer.decode(input_tokens.squeeze(0)[-num_words:], skip_special_tokens=True)

# Example usage (batch size = 1)
for batch_idx, (x, y) in enumerate(test_dataloader):
    x = x[:1].to(device)  # Take only one sentence (batch size = 1)
    output_text = generate_text(model, tokenizer, x, num_words=6)
    print("\nFinal Output:\n", output_text)
    break



Final Output:
 . Tim did not like being wet, so he went back inside his house. His mom saw his wet vest and told him to take it off.

 (pred started here) Timmy's mommy said
